In [1]:
import numpy as np
from StringIO import StringIO
import pandas as pd
from __future__ import division
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import scipy as sp
import datetime
import math
import operator
import re
import os
import os.path
import urllib, json
import csv
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import LeavePLabelOut
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request
from operator import itemgetter
from scipy.spatial import KDTree
from numpy import linalg as LA
from calendar import monthrange
from sklearn import metrics
import tarfile
import dask

#from deepmoji.sentence_tokenizer import SentenceTokenizer
#from deepmoji.model_def import deepmoji_emojis
#from deepmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
#from deepmoji.model_def import deepmoji_transfer
#from deepmoji.global_variables import PRETRAINED_PATH
#from deepmoji.finetuning import (
#    load_benchmark,
#    finetune)

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import dask.multiprocessing
from dask import compute, delayed

import time
%matplotlib inline

/home/s132732/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [76]:
YEAR="2016"

In [77]:
emoji_df = pd.read_csv(YEAR+"_emo.out",
                 sep=",", #or delim_whitespace=True, #separator is whitespace
                 header=None, #no header
                 names=['JOY','FEAR','ANGER','SADNESS','DISGUST','SHAME','GUILT'] #set columns names joy, fear, anger, sadness, disgust, shame, guilt
                 ) #parse datetime

In [78]:
emoji_df.shape

(5903755, 7)

In [79]:
ill_df=pd.read_csv('data/US_2016_integrated.csv',header=0,encoding='utf-8', index_col=False) 
YEAR=2016
#ill_df['timestamp']=pd.to_datetime(ill_df['timestamp'], errors='coerce')
ill_df.shape

(5903755, 18)

In [80]:
twe_emo_df = pd.concat([ill_df, emoji_df], axis=1)

In [81]:

pixelwidth=0.0416666666667 #PRISM cell size (800m)
width_50=pixelwidth*12.5
width_25=pixelwidth*6.25
### BOunding box for Illinois
westlon=-124.7844079
southlat=24.7433195
eastlon=-66.9513812
northlat=49.3457868

resol_lon= int(round(abs(westlon-eastlon)/width_25))
resol_lat= int(round(abs(southlat-northlat)/width_25))

n_lat, n_lon=(resol_lat, resol_lon)
lat_bins = np.linspace(southlat, northlat, n_lat)
lon_bins = np.linspace(westlon, eastlon, n_lon)

lat_labels=np.arange(resol_lat-1)
lon_labels=np.arange(resol_lon-1)

### Bin all observations
twe_emo_df = twe_emo_df.assign(
    LAT_cut=pd.cut(ill_df.LAT,lat_bins,labels=False),
    LON_cut=pd.cut(ill_df.LON,lon_bins,labels=False)
)

twe_emo_df=twe_emo_df.rename(index=str, columns={"count": "count_daily"})
twe_emo_df['timestamp']=pd.to_datetime(twe_emo_df['timestamp'], errors='coerce')
### Categorize on Urban or NOt
twe_emo_df.dropna(axis=0, how='any',inplace=True)
ill_urban_df=twe_emo_df[twe_emo_df['URBAN']==True]
ill_rural_df=twe_emo_df[twe_emo_df['URBAN']==False]


print "Spatial Bin on LAT/LON with resolution (n_lat, n_lon):",(resol_lat, resol_lon)
#print lat_bins
#print lon_bins
twe_emo_df.head()

Spatial Bin on LAT/LON with resolution (n_lat, n_lon): (94, 222)


,Unnamed: 0,count_daily,LAT,length_daily,LON,timestamp,txt,URBAN,TMAX,TMIN,...,length_norm,JOY,FEAR,ANGER,SADNESS,DISGUST,SHAME,GUILT,LAT_cut,LON_cut
0,1,1,25.966147,10,-80.165642,2016-01-01,"[#zordon, better, get, his, uglass, on, @, mia...",True,28.700001,23.840000,...,0.515391,0.020111,0.067347,0.343445,0.010204,0.417175,0.078778,0.062939,4.0,170.0
1,2,1,26.103777,11,-80.263008,2016-01-01,"[hows, my, hung, overs, doing, d, @, fort, lau...",True,29.049999,22.620001,...,0.566930,0.475589,0.094296,0.077437,0.031034,0.079223,0.118635,0.123786,5.0,170.0
2,3,1,27.335461,10,-82.543953,2016-01-01,"[hey, #happynewyear, eh, jessesimms, johnniewa...",True,27.379999,19.490000,...,0.515391,0.242336,0.112399,0.161188,0.082665,0.116681,0.117640,0.167090,9.0,161.0
3,4,1,28.566667,8,-81.205108,2016-01-01,"[#assistfiredept, at, 11801-12022, e, colonial...",True,28.639999,20.299999,...,0.412313,0.236974,0.249989,0.122633,0.067263,0.126259,0.084226,0.112656,14.0,166.0
4,5,1,28.577684,8,-81.465520,2016-01-01,"[#trespasser, at, 6010-6299, silver, star, rd,...",True,28.670000,20.340000,...,0.412313,0.370545,0.250524,0.086282,0.049004,0.098385,0.063905,0.081355,14.0,165.0


In [82]:
ill_urban_monthlygrouped_df=ill_urban_df.groupby([ill_urban_df.month,ill_urban_df.LAT_cut,ill_urban_df.LON_cut])['count_daily'].sum().to_frame().reset_index()
ill_rural_monthlygrouped_df=ill_rural_df.groupby([ill_rural_df.month,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

ill_urban_monthlygrouped_df=ill_urban_monthlygrouped_df.rename(index=str, columns={"count_daily": "monthly_cell_count_urban"})
ill_rural_monthlygrouped_df=ill_rural_monthlygrouped_df.rename(index=str, columns={"count_daily": "monthly_cell_count_rural"})

ill_urban_monthlygrouped_df['monthly_cell_daily_count'] = ill_urban_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_urban'] / monthrange(YEAR,int(row.month))[1], axis=1)
ill_rural_monthlygrouped_df['monthly_cell_daily_count'] = ill_rural_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_rural'] / monthrange(YEAR,int(row.month))[1], axis=1)

ill_merge_monthlygrouped_df=pd.merge(ill_urban_monthlygrouped_df, ill_rural_monthlygrouped_df, how='outer', on=['month','LAT_cut','LON_cut'], sort=True,
         suffixes=('_urban', '_rural'))

ill_merge_monthlygrouped_df.fillna(0).head() ##MONTHLY DAILY CELL COUNT

,month,LAT_cut,LON_cut,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,1,0.0,168.0,51.0,1.645161,17.0,0.548387
1,1,0.0,169.0,5.0,0.161290,0.0,0.000000
2,1,1.0,169.0,73.0,2.354839,6.0,0.193548
3,1,2.0,168.0,0.0,0.000000,9.0,0.290323
4,1,2.0,169.0,153.0,4.935484,21.0,0.677419


In [83]:
ill_urban_dailygrouped_df=ill_urban_df.groupby([ill_urban_df.timestamp,ill_urban_df.LAT_cut,ill_urban_df.LON_cut]).agg({'count_daily':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut]).agg({'count_daily':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
#ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

ill_merge_dailygrouped_df=pd.merge(ill_urban_dailygrouped_df, ill_rural_dailygrouped_df, how='outer', on=['timestamp','LAT_cut','LON_cut','month'], sort=True, suffixes=('_urban', '_rural'))

ill_merge_daily_df=pd.merge(ill_merge_dailygrouped_df,ill_merge_monthlygrouped_df, how="left", on=['LAT_cut','LON_cut','month'])

ill_merge_daily_df=ill_merge_daily_df
#f, ax = plt.subplots(1, 1, sharey=True)
#f.set_size_inches(18.5, 10.5, forward=True)

#test=ill_merge_daily_df[ill_merge_daily_df['timestamp']=='2016-02-21']
#plt.scatter(test['LON_cut'], test['LAT_cut'], s=700, c=test['PRCP_rural'], cmap="Reds", alpha=0.5,edgecolors=None,linewidths=0)
ill_merge_daily_df['timestamp']=ill_merge_daily_df['timestamp'].dt.strftime("%Y-%m-%d")
ill_merge_daily_df.head()

,timestamp,LAT_cut,LON_cut,TMIN_urban,CC_urban,month,count_daily_urban,SADNESS_urban,GUILT_urban,DISGUST_urban,...,TMAX_rural,SHAME_rural,HUM_rural,ANGER_rural,PRCP_rural,WIND_rural,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,2016-01-01,0.0,168.0,25.090000,70.899902,1,1.0,0.030871,0.102387,0.047038,...,28.260000,0.083638,80.75,0.076419,0.8,2.424947,51.0,1.645161,17.0,0.548387
1,2016-01-01,1.0,169.0,24.966666,70.899902,1,6.0,0.060357,0.102970,0.159005,...,NaN,NaN,NaN,NaN,NaN,NaN,73.0,2.354839,6.0,0.193548
2,2016-01-01,2.0,169.0,21.677500,70.899902,1,4.0,0.053706,0.096448,0.157428,...,NaN,NaN,NaN,NaN,NaN,NaN,153.0,4.935484,21.0,0.677419
3,2016-01-01,3.0,169.0,22.164490,70.899902,1,57.0,0.053640,0.081890,0.161072,...,28.836666,0.050259,80.75,0.062129,0.8,2.424947,1543.0,49.774194,128.0,4.129032
4,2016-01-01,3.0,170.0,24.464121,70.899902,1,206.0,0.062874,0.092948,0.129687,...,28.802857,0.087893,80.75,0.077403,0.0,2.424947,4943.0,159.451613,67.0,2.161290


In [89]:
ill_count_daily_df=pd.read_csv('data/US_2016_25KMbinned.csv',encoding='utf-8',index_col=0)

In [90]:
print "Emo shape ", ill_merge_daily_df.shape
print "Count shape ", ill_count_daily_df.shape

Emo shape  (891806, 36)
Count shape  (891806, 31)


In [91]:
ill_count_daily_df.head()

,timestamp,LAT_cut,LON_cut,count_daily_urban,TMIN_urban,CC_urban,HUM_urban,TMAX_urban,PRCP_urban,WIND_urban,...,monthly_cell_daily_count_rural,TMAX_HIS_mean,TMIN_HIS_mean,PRCP_HIS_mean,HUM_HIS_mean,WIND_HIS_mean,CC_HIS_mean,ncount_daily_urban,ncount_daily_rural,cell_id
0,2016-01-01,0.0,168.0,1.0,25.090000,70.899902,80.75,28.260000,0.800000,2.424947,...,0.548387,NaN,NaN,NaN,80.268669,7.187416,57.916634,-0.645161,0.451613,"(0.0, 168.0)"
1,2016-01-01,1.0,169.0,6.0,24.966666,70.899902,80.75,28.675000,0.098333,2.424947,...,0.193548,24.545000,18.684334,2.985333,80.268669,7.187416,57.916634,3.645161,NaN,"(1.0, 169.0)"
2,2016-01-01,2.0,169.0,4.0,21.677500,70.899902,80.75,29.045000,1.002500,2.424947,...,0.677419,25.530334,16.049334,0.603667,80.268669,7.187416,57.916634,-0.935484,NaN,"(2.0, 169.0)"
3,2016-01-01,3.0,169.0,57.0,22.164490,70.899902,80.75,28.725306,0.797551,2.424947,...,4.129032,25.593000,14.917002,1.194667,80.268669,7.187416,57.916634,7.225806,-1.129032,"(3.0, 169.0)"
4,2016-01-01,3.0,170.0,206.0,24.464121,70.899902,80.75,28.550545,0.005333,2.424947,...,2.161290,24.871666,16.777670,1.140000,80.268669,7.187416,57.916634,46.548387,5.838710,"(3.0, 170.0)"


In [92]:
cols_to_use = ill_merge_daily_df.columns.difference(ill_count_daily_df.columns)
cols_to_use = cols_to_use.tolist()+ ['timestamp', 'LAT_cut','LON_cut']
print cols_to_use

['ANGER_rural', 'ANGER_urban', 'DISGUST_rural', 'DISGUST_urban', 'FEAR_rural', 'FEAR_urban', 'GUILT_rural', 'GUILT_urban', 'JOY_rural', 'JOY_urban', 'SADNESS_rural', 'SADNESS_urban', 'SHAME_rural', 'SHAME_urban', 'timestamp', 'LAT_cut', 'LON_cut']


In [93]:
ill_emo_count_daily_df = pd.merge(ill_count_daily_df, ill_merge_daily_df[cols_to_use], how='inner', on=['timestamp', 'LAT_cut','LON_cut'],validate="one_to_one")

In [94]:
print "Merge Shape ",ill_emo_count_daily_df.shape

Merge Shape  (891806, 45)


In [95]:
ill_emo_count_daily_df.to_csv('data/US_2016_EMO_25KMbinned.csv',encoding='utf-8')